In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

In [2]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [3]:
driver.get("https://ev.or.kr/nportal/buySupprt/initSubsidyPaymentCheckAction.do")

In [4]:
#driver.find_element(By.CSS_SELECTOR,"#btnLocalCarPrc").click()

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [5]:
wait = WebDriverWait(driver, 10)
element = driver.find_element(By.CSS_SELECTOR, '[id="btnLocalCarPrc"]')

In [6]:

# 현재 창 handle 저장
main_window = driver.current_window_handle

In [7]:
# 어떤 요소 클릭 시 새 창이 열림
driver.find_element(By.ID, "btnLocalCarPrc").click()

In [8]:
# 새 창 handle로 전환
for handle in driver.window_handles:
    if handle != main_window:
        driver.switch_to.window(handle)
        break

In [9]:
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time

전체_데이터 = []  # 모든 지역 데이터 저장 리스트

buttons = driver.find_elements(By.CLASS_NAME, "btnDown")
조회_버튼들 = [btn for btn in buttons if btn.text.strip() == "조회"]

for i in range(len(조회_버튼들)):
    # 버튼 다시 찾기 (페이지 이동 후 DOM이 변하기 때문)

    buttons = driver.find_elements(By.CLASS_NAME, "btnDown")
    조회_버튼들 = [btn for btn in buttons if btn.text.strip() == "조회"]

    # 지역명 추출 (HTML 구조에 따라 조정)
    지역명 = buttons[i].find_element(By.XPATH, "./..").text.strip()

    buttons[i].click()


    driver.switch_to.window(handle)
        

    # 테이블 로딩 대기 (ID, 클래스명 등 실제 데이터 영역에 맞게 수정)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.TAG_NAME, "table"))
    )

    # 테이블 찾기 (ID나 클래스명 있으면 바꾸세요)
    table = driver.find_element(By.CSS_SELECTOR, "table.table01.fz15")

    # 테이블의 모든 행 가져오기 (헤더 제외)
    rows = table.find_elements(By.XPATH, "./tbody/tr")

    for row in rows:
        cols = row.find_elements(By.TAG_NAME, "td")
        if len(cols) < 6:  # 열 개수가 부족하면 건너뜀
            continue

        데이터_딕셔너리 = {
            "지역명": 지역명,
            "차종": cols[0].text.strip(),
            "제조사": cols[1].text.strip(),
            "모델명": cols[2].text.strip(),
            "국비(만원)": cols[3].text.strip(),
            "지방비(만원)": cols[4].text.strip(),
            "보조금(만원)": cols[5].text.strip()
        }
        전체_데이터.append(데이터_딕셔너리)

    # 뒤로 가기
    driver.back()

    # 버튼 다시 로딩 대기
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "btnDown"))
    )

# 데이터프레임으로 변환 후 CSV 저장
df = pd.DataFrame(전체_데이터)
df.to_csv("지역별_차량_보조금_데이터.csv", index=False, encoding='utf-8-sig')

print("모든 지역 데이터 수집 및 CSV 저장 완료!")


KeyboardInterrupt: 

In [10]:
import pymysql
conn = pymysql.connect(host='127.0.0.1', user='ndhyua', passwd='123', database='sk15', port=3306)

In [ ]:
import pymysql
import pandas as pd

def to_number(x):
    if isinstance(x, str):
        x = x.replace(',', '').replace('만원', '').strip()
    try:
        return float(x)
    except:
        return 0

# DB 연결
conn = pymysql.connect(
    host='127.0.0.1',
    user='ndhyua',
    passwd='123',
    database='sk15',
    port=3306,
    charset='utf8mb4'
)

try:
    cursor = conn.cursor()

    # CSV 데이터 읽기
    df = pd.read_csv('subsidy_data.csv', encoding='utf-8-sig')

    # 문자열 -> 숫자 변환
    df['국비(만원)'] = df['국비(만원)'].apply(to_number)
    df['지방비(만원)'] = df['지방비(만원)'].apply(to_number)
    df['보조금(만원)'] = df['보조금(만원)'].apply(to_number)

    sql = """
    INSERT INTO subsidy_data (region, vehicle_type, manufacturer, model_name, national_subsidy, local_subsidy, total_subsidy)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    """

    for idx, row in df.iterrows():
        cursor.execute(sql, (
        row['지역명'],          # CSV 한글 컬럼명에서 값을 읽어서 영어 컬럼명에 넣음
        row['차종'],
        row['제조사'],
        row['모델명'],
        row['국비(만원)'],
        row['지방비(만원)'],
        row['보조금(만원)']
    )) 



    conn.commit()
    print("데이터 삽입 완료!")

except Exception as e:
    print("오류 발생:", e)
    conn.rollback()

finally:
    cursor.close()
    conn.close()
